before running this, you should run make_tagcomplete_csv.ipynb

In [ ]:
# running this once for the first time.
!grep ",1,[0-9]" outputs/animagine-xl-3.0.csv > intermediates/artist-animagine-xl-3.0.csv

In [ ]:
import re
import sys

LIMIT_COUNT = 150

# make artist_dict from artist csv.
file = "intermediates/artist-animagine-xl-3.0.csv"
artist_dict = {}
with open(file, 'r') as infile:
    for row in infile:
        artist, _, count, _ = row.split(',')
        if (int(count) >= LIMIT_COUNT):
            artist_dict[artist] = {}
infile.close()
artist_list = list(artist_dict.keys())

In [ ]:
unnecessary_tag_pattern = [
    '_quality',
    'commentary',
    'translat',
    '_background',
    'looking_'
]

unnecessary_tags = [
    'highres',
    'solo',
    'newest',
    'late',
    'mid',
    'absurdres',
    'nsfw',
    'comic',
    'upper_body',
    'full_body',
    'standing',
    'bad_id',
    'bad_pixiv_id',
    'bad_twitter_id',
    'sweat',
    'hetero',
    'censored',
    'indoor',
    'outdoor',
    'smile',
    'blush'
]

# read data from csv
file = "intermediates/taglist.csv"
with open(file, 'r') as infile:
    for row in infile:
        tags = row.strip().split(', ')
        artists = [];
        after_rating = False;
        for tag in tags:
            tag = tag.replace('||| ', '').strip().replace(' ', '_')

            if not tag:
                print ("got empty tag from ", tags)
                continue
            
            if tag in artist_list:
                artists.append(tag)
            elif after_rating:
                if (not any(pattern in tag for pattern in unnecessary_tag_pattern) and
                    not any(unnecessary_tag == tag for unnecessary_tag in unnecessary_tags)):
                    for artist in artists:
                        if tag not in artist_dict[artist]:
                            artist_dict[artist][tag] = 0
                        artist_dict[artist][tag] += 1
            
            if "rating:" in tag:
                after_rating = True
infile.close()

In [ ]:
TOTAL_ADDITIONAL_TAGS = 10

cut_dict = {}
for key, value in artist_dict.items():
    ordered_value = dict(sorted(value.items(), key=lambda x: x[1], reverse=True))
    cut_dict[key] = dict(list(ordered_value.items())[:TOTAL_ADDITIONAL_TAGS])

newfile = "outputs/animagine_xl_3.0_artists.txt"
with open(newfile, 'w') as outfile:
    for artist in artist_list:
        formatted_artist = artist.replace('(', '\\(').replace(')','\\)').replace('_', ' ')
        outfile.write(f"{formatted_artist}")
        for key in cut_dict[artist].keys():
            formatted = key.replace('(', '\\(').replace(')','\\)').replace('_', ' ')
            outfile.write(f", {formatted}")
        outfile.write("\n")
outfile.close()